In [14]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [4]:
# Driver's path
path = 'C:/Users/papan/Downloads/chromedriver_win32'
driver = webdriver.Chrome(path)
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)
# Enter to the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)
# Accept cookies
driver.find_element('xpath','//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[1]').click()
# User Credentials
# Reading txt file where we have our user credentials
with open('user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
user_name = user_credentials[0] # First line
password = user_credentials[1] # Second line
driver.find_element('xpath','//*[@id="username"]').send_keys(user_name)
driver.find_element('xpath','//*[@id="password"]').send_keys(password)
time.sleep(1)
# Login button
driver.find_element('xpath','//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)
# if driver.find_element('xpath','//*[@id="ember7"]/h2').text == 'Confirm your account information':
#     driver.find_element('xpath','//*[@id="ember20"]/button[1]').click()
driver.find_element('xpath','//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(5)
driver.execute_script("return window.stop();")
jobs_block = driver.find_element(By.ID,'jobs-home-vertical-list__entity-list')
jobs_list = jobs_block.find_elements(By.TAG_NAME, 'li')
driver.implicitly_wait(10)
links = []
links = []
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.TAG_NAME, "a")))
for job in jobs_list:
    all_links = job.find_elements(By.CLASS_NAME,'ember-view job-card-container__link job-card-list__title')
    for a in all_links:
            links.append(a.get_attribute('href'))

C:\Users\papan\AppData\Local\Temp\ipykernel_12852\3764508557.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [36]:
for job in jobs_list:
    innerHTML=job.get_attribute('innerHTML')
    soup = BeautifulSoup(innerHTML,'html')
    link = soup.find('a',{'class':'ember-view job-card-container__link job-card-list__title'},href=True)
    if link:
        links.append(link['href'])


<a aria-label="Data Scientist" class="ember-view job-card-container__link job-card-list__title" data-control-id="LLv1js07PQonzwCnoqfziQ==" href="/jobs/collections/recommended/?currentJobId=3600782084&amp;eBP=JYMBII_JOBS_HOME_ORGANIC&amp;refId=km3RLIu%2BX9gxtb2dPlAntg%3D%3D&amp;trackingId=LLv1js07PQonzwCnoqfziQ%3D%3D" id="ember430" tabindex="0">
              Data Scientist
            </a>
<a aria-label="Data Scientist" class="ember-view job-card-container__link job-card-list__title" data-control-id="LLv1js07PQonzwCnoqfziQ==" href="/jobs/collections/recommended/?currentJobId=3600782084&amp;eBP=JYMBII_JOBS_HOME_ORGANIC&amp;refId=km3RLIu%2BX9gxtb2dPlAntg%3D%3D&amp;trackingId=LLv1js07PQonzwCnoqfziQ%3D%3D" id="ember430" tabindex="0">
              Data Scientist
            </a>
None
None
None
None
None
None
<a aria-label="Central Monitor Data Analyst--Associate, Risk Based Monitoring" class="ember-view job-card-container__link job-card-list__title" data-control-id="HLeetYIVp4WFUvmZDCi9Yw=

In [39]:
len(jobs_list)

11